# Linear Algebra in OSCAR

* Authors: Martin Bies, Aaruni Kaushik
* Version: OSCAR version 1.2.2 or newer.

This tutorial provides an introduction to linear algebra in OSCAR. The aim is to describe important functionality and to exemplify it, so that users can start to use OSCAR more easily. Still, let us note that this tutorial does not aim to cover all of the existing functionality. Instead, we present a selection of arguably very important functions and features. For a more complete overview, please consult the documentation at https://docs.oscar-system.org/stable/LinearAlgebra/intro/.

For this tutorial, the reader need not be familiar with OSCAR, but is assumed to have some basic understanding of Julia.

The content of this notebook is as follows:
1. [Creating Matrices](#1-Creating-Matrices)
2. [Solving Linear Equation Systems](#2-Solving-Linear-Equation-Systems)
3. [Eigenvalues and Eigenspaces](#3-Eigenvalues-and-Eigenspaces)
4. [Normalforms](#4-Normalforms)
5. [Vector](#5-Vector)
6. [MatricesSpaces](#6-Matrix-Spaces)


In [1]:
using Oscar

In [2]:
Oscar.versioninfo()

OSCAR version 1.2.2
  combining:
    AbstractAlgebra.jl   v0.44.4
    GAP.jl               v0.13.0
    Hecke.jl             v0.35.8
    Nemo.jl              v0.48.1
    Polymake.jl          v0.11.24
    Singular.jl          v0.24.1


## 1 Creating Matrices

Oscar has its own type of matrix independent of Julia's in-built matrices. This is due to technical limitations in Julia.

The central reason is that every matrix should have a unique coefficient ring assigned and for a Julia matrix this is not possible. Types (even with parameters) are not suitable for this purpose. For details, see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix_algebras/#Basic-matrix-functionality.

Oscar matrices are typically created with the (Oscar) function `matrix` with a small "m"; whereas Julia's build-in matrices have types written `Matrix{...}` with a big "M". In conclusion, use `matrix` with small "m" within OSCAR.

In the following, we will elaborate on how we create such OSCAR matrices. For conversion of the "normal" julia matrices into OSCAR matrices, see for instance https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Conversion-to-Julia-matrices,-iteration-and-broacasting.

There are many ways to create matrices in $\texttt{OSCAR}$. We demonstrate a few.

In constrast to ordinary Julia matrices, please note that we specify the specify the coefficients ring.

In [3]:
m = matrix(ZZ, [1 2 3 ; 4 5 6])

[1   2   3]
[4   5   6]

Note that the above secretly creates a Julia matrix and then turns it into an OSCAR matrix over the ring $ZZ$. This is the julia-ish way of creating such matrices.

However, in practical applications, you might find this syntax not very human-readable. Eyeball the following:

In [4]:
R, x = polynomial_ring(ZZ, :x)
matrix(R, [x^2 -x-1; x^2-x -1])

[    x^2   -x - 1]
[x^2 - x       -1]

Here are alternatives to create the above matrix $m$, but such that they syntax is readable even in involved settings such as polynoimal entries:

In [5]:
m = matrix(ZZ, [[1,2,3], [4,5,6]])
m = matrix(ZZ, 2,3, [1,2,3,4,5,6])

[1   2   3]
[4   5   6]

We can create a matrix with the same coefficients over the rationals:

In [6]:
m2 = matrix(QQ, [1 2 3; 4 5 6])

[1   2   3]
[4   5   6]

Note that these two matrices print the same but they have different types.

In [7]:
typeof(m)

ZZMatrix

In [8]:
typeof(m2)

QQMatrix

Despite these matrices being defined over different coefficient rings, we can (for instance) add them:

In [9]:
m+m2

[2    4    6]
[8   10   12]

In [10]:
base_ring(ans)

Rational field

You will likely not use the function base_ring too often. Still, it can be rather useful in order to identify over which ring a matrix is defined, for instance when (trying to) add or compare two matrices.

In the case at hand, just as addition was possible above, so is comparison:

In [11]:
m == m2

true

This fails for other coefficient rings. As an example, let us consider the following matrices:

In [12]:
F9, a = finite_field(9, "a")
I_f9 = identity_matrix(F9, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

In [13]:
I_zz = identity_matrix(ZZ, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

In [14]:
I_qq = identity_matrix(QQ, 4)

[1   0   0   0]
[0   1   0   0]
[0   0   1   0]
[0   0   0   1]

Note that we used a convenience functions to create these identity matrices. More such functionality does exist, see e.g. https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Basic-matrix-functionality. Here is another example use of such a convenience function:

In [15]:
zero_matrix(ZZ, 4, 5)

[0   0   0   0   0]
[0   0   0   0   0]
[0   0   0   0   0]
[0   0   0   0   0]

Let us now return to our matrices $I_{zz}$, $I_{qq}$ and $I_{f9}$. Those are defined over different coefficient rings. While they print the same, not all pairs of them can be compared:

In [16]:
I_zz == I_qq

true

In general, elements of different ring may not be compared nor combined. And this extend to matrices over different rings. However, if there is a unique embedding of one ring into the other, then OSCAR will do this for us automatically. Thereby, sometimes matrices over different rings can be compared, and sometimes not. In the case at hand, $I_{f9}$ and $I_{qq}$ cannot be compared, but $I_{zz}$ and $I_{f9}$ can:

In [17]:
I_zz == I_f9

true

## 2 Solving Linear Equation Systems

Let us solve the following system of linear equations in $\texttt{OSCAR}$ over $\mathbb{Q}$:

$$ \left[ \begin{array}{ccc} 1 & 2 & 3 \\ 2 & 4 & 6 \end{array} \right] \cdot v = \left[ \begin{array}{c} -5 \\ -10 \end{array} \right] $$

In [18]:
m = matrix(QQ, [[1,2,3], [2,4,6]])

[1   2   3]
[2   4   6]

In [19]:
b = matrix(QQ, [[-5], [-10]])

[ -5]
[-10]

In [20]:
solve(m, b; side = :right)

[-5]
[ 0]
[ 0]

Side comment: The semicolon separates the mandatory arguments in the beginning from optional keyword argument.

But of course, this is merely a single solution. To obtain all solutions, we need to add the nullspace.

In [21]:
d, nb = nullspace(m)

(2, [-2 -3; 1 0; 0 1])

The first return value is the dimension of the nullspace, and the second is a basis of the nullspace.

In [22]:
nb

[-2   -3]
[ 1    0]
[ 0    1]

Additional information is available in the OSCAR documentation. See for instance:
* https://docs.oscar-system.org/stable/AbstractAlgebra/linear_solving/#solving_chapter.
* https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#nullspace-Union{Tuple{MatElem{T}},%20Tuple{T}}%20where%20T%3C:FieldElem

## 3 Eigenvalues and Eigenspaces

In this section, we demonstrate how to find eigenvalues, and eigenspaces of a diagonalizable matrix.
For simplicity, we focus on the case of a matrix that is diagonalizable over $\mathbb{Q}$

In [23]:
M = matrix(QQ, [[-16, -28, -19], [42, 69, 46], [-42, -72, -49]])

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [24]:
det(M)

-30

In [25]:
characteristic_polynomial(M)

x^3 - 4*x^2 - 11*x + 30

The eigenvalues are zeros of the characteristic polynomial. In the case above, we are working with a matrix over $\mathbb{Q}$. So the zero of the characteristic polynomial are elements of the algebraic closure of $\mathbb{Q}$. For a field $F$ other than $\mathbb{Q}$, the zeros of the characteristic polynomial are elements of the algebraic closure of said field $F$. Since OSCAR covers a large variety of fields, the eigenvalue function must be supplied with the field in which the eigenvalues will reside:

In [26]:
eigenvalues(M)

3-element Vector{QQFieldElem}:
 2
 5
 -3

In [27]:
eigenspace(M, QQ(2))

[7   4   1]

In [28]:
A = vcat([eigenspace(M, k) for k in eigenvalues(M)]...)

[7   4   1]
[2   2   1]
[0   1   1]

In [29]:
D = diagonal_matrix(QQ, eigenvalues(M))

[2   0    0]
[0   5    0]
[0   0   -3]

In [30]:
(inv(A) * D * A) == M

true

For more details see e.g. https://docs.oscar-system.org/stable/Nemo/algebraic/#eigenvalues-Tuple{QQBarField,%20ZZMatrix}.

## 4 Normalforms

There are many many normal forms, we demonstrate how to compute the Smith and the Hermite normal form.

### 4.1 Smith Normal Form

In [31]:
M = matrix(ZZ, M)

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [32]:
base_ring(M)

Integer ring

In [33]:
snf(M)

[1   0    0]
[0   1    0]
[0   0   30]

In [34]:
S, T, U = snf_with_transform(M)

([1 0 0; 0 1 0; 0 0 30], [13 -3 -8; -18 4 11; -69 16 43], [-2 -4 19; 1 3 -16; 0 -1 6])

In [35]:
T * M * U == S

true

 For more details see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Smith-normal-form.

### 4.2 Hermite Normal Form

In [36]:
M

[-16   -28   -19]
[ 42    69    46]
[-42   -72   -49]

In [37]:
base_ring(M)

Integer ring

In [38]:
hnf(M)

[2   2   4]
[0   3   3]
[0   0   5]

In [39]:
H, U = hnf_with_transform(M)

([2 2 4; 0 3 3; 0 0 5], [13 -2 -7; 0 -1 -1; 21 -4 -12])

In [40]:
U * M == H

true

For more details see https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Hermite-normal-form.

## 5 Vector

One way of creating vectors, is by extracting rows and columns of a matrix. Let us demonstrate this:

In [41]:
M = matrix(QQ, [[1,2,3], [4,5,6]])

[1   2   3]
[4   5   6]

In [42]:
row = M[1:1,:]

[1   2   3]

In [43]:
col = M[:,1:1]

[1]
[4]

Certainly, we can multiply the original matrix from the right with the transpose of row:

In [44]:
M * transpose(row)

[14]
[32]

Likewise, we can multiply the original matrix from the left with the transpose of col:

In [45]:
transpose(col) * M

[17   22   27]

Here is alternative way of extracting the first row, so that you can multiply it (secretly being treated as a column vector) from the right with the matrix $M$, but without having to use the method "transpose":

In [46]:
v = M[1,:]

3-element Vector{QQFieldElem}:
 1
 2
 3

In [47]:
M*v

2-element Vector{QQFieldElem}:
 14
 32

Do not be confused. While we extract the first row of our matrix $M$, julia prints it as a column, simply because that is the default for any list of elements (the data structure of which is called "Vector" in julia, to add to the confusion). Still, we extracted the first row of the matrix.

# 6 Matrix Spaces

Given an $r \times c$ matrix $A$, then it is an element of the set of all $r \times c$ matrices over the base ring of $A$. The collection of these matrices is called a matrix space. For functionality of this within OSCAR, see for instance https://docs.oscar-system.org/stable/AbstractAlgebra/matrix/#Matrix-functionality.
